영문 원본 LLAVA - 허깅페이스에서 가져와서 pip 받을게 거의 없음

In [4]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# 모델과 프로세서 로드
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# 이미지 불러오기
image = Image.open('./testimage/test.png')

# 이미지를 pixel_values로 변환 (전처리)
inputs = processor(image, return_tensors="pt")

# 캡션 생성
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0], skip_special_tokens=True)

print("Generated caption:", caption)


C:\Sorigil-AI\.venv\Lib\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated caption: araffe cat with a piece of bread on its head


OCR

In [6]:
import easyocr

# EasyOCR 리더 초기화 (한글과 영어 지원)
reader = easyocr.Reader(['ko', 'en'])

# OCR 수행
result = reader.readtext('/testimage/test6.png')
text = '\n'.join([res[1] for res in result])

print("Generated OCR text:", text)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Generated OCR text: 2.44
al LTE) <
103
D2 파일이 공지로 등록되없습니다
Vv 할 일 7H
예약 메시지
린아이에거 마시델로 1개틀 주고 15
안 먹지 안고 참으면 2개틀 주기로
올 때, 먹지
참아서 2개틀 받은
아이들이 커서 그렇지 않앗면 아이들 보
더 훌륭하게 되없다는 심리학 실험인   다시델로
야기
있다. 미국에서 이
험에 참가한 호아침 데 포사다는 같은 이름의
자기계발서도 벗다. 하지만 이 -
대해서 논란이
이틀데면
2013년 로체스터
대달표데 
돌리 팔데리
와 리처드
어측린은 구월_Coqnition-인 "Rational 508
Olooog"o
제목으로 발표한
문에서 이 연
결과에
표하여 "첫
눈문예 액물굴
빨리 덕은 아이들 중 일부는 참을
부 즉햇단 것 이
'나중에 들아오면 하나들 터 주컷다
말을 의심
'따문이라고 한다. 그들은
안정 한 환경에서 자란 아이들은
먹는 것이 남는
이라는 생각올 갖게 된다"눈 말 올 남기다, "안정적인
자란 아 이들
일주록 약속이 뒷켜질 젓이라고
대하여 좀 더
오래
오전 8.13
오전 9.51
안고
라는
성이


ocr, 캡션 합치기

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import easyocr
import numpy as np

# BLIP 모델과 프로세서 로드
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# EasyOCR 리더 초기화 (한글과 영어 지원)
ocr_reader = easyocr.Reader(['ko', 'en'])

# 이미지 불러오기
image = Image.open('./testimage/test6.png').convert('RGB')

# 1. EasyOCR을 사용하여 이미지 내 텍스트 추출 (PIL 이미지를 numpy 배열로 변환)
ocr_result = ocr_reader.readtext(np.array(image))
ocr_text = ' '.join([res[1] for res in ocr_result])

# 2. BLIP를 사용하여 이미지 캡션 생성
inputs = blip_processor(image, return_tensors="pt")
outputs = blip_model.generate(**inputs)
caption = blip_processor.decode(outputs[0], skip_special_tokens=True)

# 3. OCR 결과와 BLIP 캡션 결합
combined_caption = f"OCR Text: {ocr_text}\nGenerated Caption: {caption}"

print(combined_caption)


한글 llava 시도

In [0]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCausalLM

processor = AutoProcessor.from_pretrained("KBNIT/KoLLaVA-1.5v-kolon-v1.6")
model = AutoModelForCausalLM.from_pretrained("KBNIT/KoLLaVA-1.5v-kolon-v1.6")

# 이미지 불러오기
image = Image.open('test4.png')

# 이미지를 pixel_values로 변환 (전처리)
inputs = processor(image, return_tensors="pt")

# 캡션 생성
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0], skip_special_tokens=True)

print("Generated caption:", caption)

허깅페이스 api 호출

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수에서 Hugging Face API 토큰 가져오기
api_token = os.getenv('HUGGING_FACE_API')

In [3]:
import requests
from PIL import Image
import base64

# 이미지 파일을 읽어서 base64로 변환
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Hugging Face API 호출 함수
def query(image_path):
    API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
    headers = {"Authorization": f"Bearer {api_token}"}  # Hugging Face API 토큰
    image_data = encode_image(image_path)

    # API 요청 데이터 (이미지를 base64로 변환 후 전송)
    payload = {
        "inputs": {
            "image": image_data
        }
    }

    # API 호출
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 이미지 파일 경로
image_path = 'test4.png'

# 결과 출력
result = query(image_path)
print("Generated caption:", result)


Generated caption: [{'generated_text': 'a close up of a cell phone with a text message'}]
